In [8]:
import joblib
import itertools as it

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.style.use('seaborn-dark-palette')

font = {'family': 'normal',
        'weight': 'normal',
        'size'  : 16}
plt.tight_layout()

mpl.rc('font', **font)

sns.set_style("whitegrid")

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

pd.options.display.float_format = '{:,.4f}'.format

## Load wind power data

In [9]:
df_wind_power = pd.read_csv('../data/wind/wind_power_train.csv',
                            header=0,
                            index_col=0,
                            parse_dates=True)

In [10]:
df_wind_meteo = pd.read_csv('../data/wind/wind_meteo_train.csv',
                            header=[0, 1],
                            skiprows=[2],
                            index_col=0,
                            parse_dates=True)

In [11]:
df_wind_power.head()

,1,2,3,4,5,6,7,8,9,10
TIMESTAMP,,,,,,,,,,
2012-01-01 01:00:00,0.0000,0.5963,0.4255,0.3782,0.2737,0.2681,0.0000,0.0000,0.0000,0.5944
2012-01-01 02:00:00,0.0549,0.4112,0.3639,0.0630,0.0868,0.0345,0.0147,0.0150,0.0000,0.5697
2012-01-01 03:00:00,0.1102,0.1672,0.2971,0.0367,0.0068,0.0206,0.0351,0.0718,0.0000,0.3305
2012-01-01 04:00:00,0.1651,0.0373,0.2354,0.0343,0.0186,0.0019,0.0451,0.0668,0.0000,0.2113
2012-01-01 05:00:00,0.1569,0.0512,0.1209,0.0336,0.0348,0.0152,0.0502,0.0348,0.0000,0.1721


In [12]:
df_wind_meteo.head()

1                              2                  \
                       U10     V10   U100    V100     U10     V10    U100   
2012-01-01 01:00:00 2.1246 -2.6820 2.8643 -3.6661 -0.1716 -5.4660 -0.1126   
2012-01-01 02:00:00 2.5217 -1.7970 3.3449 -2.4648 -0.0889 -4.6432 -0.0141   
2012-01-01 03:00:00 2.6722 -0.8225 3.5084 -1.2141 -0.2460 -3.6134 -0.2252   
2012-01-01 04:00:00 2.4575 -0.1436 3.2152 -0.3555 -0.6807 -2.9196 -0.7720   
2012-01-01 05:00:00 2.2459  0.3896 2.9577  0.3327 -1.2611 -2.6198 -1.4875   

                                 3                              4          \
                       V100    U10     V10   U100    V100     U10     V10   
2012-01-01 01:00:00 -7.1013 2.1932 -4.6492 3.0284 -6.2216  0.5349 -3.6602   
2012-01-01 02:00:00 -5.8968 2.9701 -3.9443 3.9811 -5.2541  0.3308 -2.6764   
2012-01-01 03:00:00 -4.4894 3.2389 -3.1445 4.3558 -4.1634 -0.0658 -2.0291   
2012-01-01 04:00:00 -3.5988 3.2610 -2.2288 4.3139 -2.9155 -0.4195 -1.7991   
2012-01-01 05:00:00 -3.2447 2.9400 -2.0401 3.9336 -2.6898 -0.7542 -1.6615   

                                          5                               6  \
                       U100    V100     U10     V10    U100    V100     U10   
2012-01-01 01:00:00  0.7654 -4.4867  0.5349 -3.6602  0.7654 -4.4867  1.2056   
2012-01-01 02:00:00  0.4699 -3.2138  0.3308 -2.6764  0.4699 -3.2138  0.9811   
2012-01-01 03:00:00 -0.0209 -2.4181 -0.0658 -2.0291 -0.0209 -2.4181  0.6133   
2012-01-01 04:00:00 -0.4449 -2.1497 -0.4195 -1.7991 -0.4449 -2.1497  0.2427   
2012-01-01 05:00:00 -0.8383 -1.9918 -0.7542 -1.6615 -0.8383 -1.9918 -0.1384   

                                                  7                        \
                        V10    U100    V100     U10    V10    U100   V100   
2012-01-01 01:00:00 -2.4179  1.4678 -2.9561 -0.3075 0.3713 -0.6254 0.2985   
2012-01-01 02:00:00 -1.5776  1.1491 -1.9371  0.1400 0.7516  0.0250 0.7975   
2012-01-01 03:00:00 -1.0112  0.6974 -1.2607  0.6761 1.4722  0.6748 1.6268   
2012-01-01 04:00:00 -0.7467  0.2621 -0.9565  0.9072 2.2588  0.9369 2.6258   
2012-01-01 05:00:00 -0.5014 -0.1675 -0.6918  0.7394 2.9079  0.7206 3.5017   

                          8                            9                 \
                        U10    V10    U100   V100    U10     V10   U100   
2012-01-01 01:00:00 -0.3075 0.3713 -0.6254 0.2985 1.9033 -1.4950 2.5167   
2012-01-01 02:00:00  0.1400 0.7516  0.0250 0.7975 2.1733 -0.6436 2.8371   
2012-01-01 03:00:00  0.6761 1.4722  0.6748 1.6268 2.3127  0.1366 3.0341   
2012-01-01 04:00:00  0.9072 2.2588  0.9369 2.6258 2.2176  0.7972 2.9418   
2012-01-01 05:00:00  0.7394 2.9079  0.7206 3.5017 1.9006  1.2857 2.5592   

                                10                         
                       V100    U10     V10   U100    V100  
2012-01-01 01:00:00 -2.0781 1.6558 -4.6496 2.0243 -5.9922  
2012-01-01 02:00:00 -1.0069 2.1786 -4.1849 2.6231 -5.3181  
2012-01-01 03:00:00 -0.0155 2.2286 -3.2424 2.6452 -4.0522  
2012-01-01 04:00:00  0.8235 2.0772 -2.3451 2.4329 -2.9138  
2012-01-01 05:00:00  1.4711 1.8147 -1.5776 2.0978 -1.9761

## Load solar power data

In [13]:
df_solar_power = pd.read_csv('../data/solar/solar_power_train.csv',
                             header=0,
                             index_col=0,
                             parse_dates=True)

df_solar_meteo = pd.read_csv('../data/solar/solar_meteo_train.csv',
                             header=[0, 1],
                             skiprows=[2],
                             index_col=0,
                             parse_dates=True)

In [14]:
df_solar_power.head()

,1,2,3
TIMESTAMP,,,
2012-04-01 01:00:00,0.7541,0.6473,0.7990
2012-04-01 02:00:00,0.5550,0.6218,0.8175
2012-04-01 03:00:00,0.4384,0.4727,0.5715
2012-04-01 04:00:00,0.1454,0.0989,0.0638
2012-04-01 05:00:00,0.1120,0.1044,0.1037


In [15]:
df_solar_meteo.head()

ZONEID,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3
,VAR134,VAR134,VAR134,VAR157,VAR157,VAR157,VAR164,VAR164,VAR164,VAR165,VAR165,VAR165,VAR166,VAR166,VAR166,VAR167,VAR167,VAR167,VAR169,VAR169,VAR169,VAR175,VAR175,VAR175,VAR178,VAR178,VAR178,VAR228,VAR228,VAR228,VAR78,VAR78,VAR78,VAR79,VAR79,VAR79
2012-04-01 01:00:00,"94,843.6250","94,290.6250","92,667.6250",60.2219,57.3743,57.0032,0.2446,0.1910,0.2381,1.0393,0.8579,0.9862,-2.5030,-2.8016,-3.1334,294.4485,294.3933,293.9036,"2,577,830.0000","2,595,302.0000","2,606,438.0000","1,202,532.0000","1,192,092.0000","1,158,284.0000","2,861,797.0000","2,884,677.0000","2,901,861.0000",0.0000,0.0000,0.0000,0.0020,0.0015,0.0035,0.0036,0.0054,0.0247
2012-04-01 02:00:00,"94,757.9375","94,217.6875","92,622.6875",54.6786,57.1298,64.9608,0.4571,0.5171,0.5616,2.4829,2.5941,2.6884,-2.9933,-2.6532,-2.5749,295.6514,294.9624,293.1965,"5,356,093.0000","5,374,973.0000","5,352,637.0000","2,446,757.0000","2,420,485.0000","2,341,093.0000","5,949,378.0000","5,984,578.0000","5,980,930.0000",0.0000,0.0000,0.0008,0.0055,0.0183,0.0275,0.0336,0.1016,0.1405
2012-04-01 03:00:00,"94,732.8125","94,201.0625","92,598.8125",61.2949,63.0761,66.7959,0.7714,0.7052,0.6616,3.3399,2.8814,2.8845,-1.9825,-1.6478,-1.8937,294.4546,293.8767,292.5925,"7,921,788.0000","7,858,492.0000","7,714,300.0000","3,681,336.0000","3,643,656.0000","3,531,304.0000","8,939,176.0000","8,929,064.0000","8,822,632.0000",0.0013,0.0013,0.0017,0.0301,0.0353,0.0324,0.1320,0.1129,0.1366
2012-04-01 04:00:00,"94,704.0625","94,156.0625","92,542.0625",67.7753,62.9755,65.1776,0.9659,0.9934,0.9416,3.1061,3.0248,3.1757,-1.4461,-1.4968,-1.6399,293.2615,293.0713,292.1748,"9,860,520.0000","9,719,720.0000","9,619,368.0000","4,921,504.0000","4,885,120.0000","4,740,096.0000","11,331,679.0000","11,258,463.0000","11,178,847.0000",0.0025,0.0017,0.0021,0.0572,0.0645,0.0320,0.1106,0.1078,0.0974
2012-04-01 05:00:00,"94,675.0000","94,124.2500","92,508.0000",70.1730,65.0538,65.7443,0.9447,0.9376,0.9447,2.6011,2.5216,2.6973,-1.9045,-1.8366,-1.8801,292.7329,292.5444,291.5845,"11,143,097.0000","11,054,009.0000","11,054,009.0000","6,254,380.0000","6,211,372.0000","6,014,412.0000","13,105,558.0000","13,079,318.0000","13,079,958.0000",0.0033,0.0021,0.0025,0.0510,0.0595,0.0485,0.1896,0.1576,0.1638


In [17]:
df_solar_power.describe()

,1,2,3
count,"15,360.0000","15,360.0000","15,360.0000"
mean,0.1735,0.1923,0.1992
std,0.2628,0.2795,0.2880
min,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000
50%,0.0027,0.0024,0.0030
75%,0.2848,0.3575,0.3707
max,0.9162,0.9778,1.0035


In [16]:
df_solar_meteo.describe()

ZONEID,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,3
,VAR134,VAR134,VAR134,VAR157,VAR157,VAR157,VAR164,VAR164,VAR164,VAR165,VAR165,VAR165,VAR166,VAR166,VAR166,VAR167,VAR167,VAR167,VAR169,VAR169,VAR169,VAR175,VAR175,VAR175,VAR178,VAR178,VAR178,VAR228,VAR228,VAR228,VAR78,VAR78,VAR78,VAR79,VAR79,VAR79
count,"15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000","15,359.0000"
mean,"94,718.3937","94,161.1335","92,512.0336",68.2732,66.9880,67.6927,0.4205,0.4243,0.4416,1.0103,0.8743,0.8528,-0.3474,-0.2465,-0.2285,285.0612,284.7675,283.9830,"12,398,208.8161","12,375,548.7599","12,336,392.7126","13,715,585.1388","13,655,192.0478","13,378,715.9572","14,371,885.9191","14,390,984.8888","14,381,453.0647",0.0009,0.0009,0.0010,0.0358,0.0371,0.0384,0.0154,0.0155,0.0159
std,602.1085,602.4702,593.2916,19.7350,19.8961,20.3269,0.3980,0.3957,0.3976,2.8104,2.4464,2.1638,1.9077,1.9664,2.2252,7.0096,6.9243,6.7524,"6,563,589.3376","6,510,942.5721","6,464,259.5407","7,653,794.9908","7,643,903.1610","7,541,980.3139","6,983,524.3402","6,975,865.8447","6,962,384.7479",0.0034,0.0033,0.0034,0.1063,0.1060,0.1079,0.0496,0.0487,0.0481
min,"92,587.8125","92,047.3750","90,359.0000",10.1523,10.1776,8.5254,0.0000,0.0000,0.0000,-8.9086,-8.8352,-8.2788,-7.6361,-7.5002,-8.4042,270.2664,269.4363,269.5964,"187,820.3750","194,134.0000","149,878.0000","865,972.8125","838,841.5625","821,617.5625","657,726.0625","709,579.0000","646,523.0000",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,"94,310.3750","93,751.7188","92,105.4062",53.5192,51.7607,52.0255,0.0080,0.0097,0.0153,-0.9151,-0.6259,-0.4029,-1.5657,-1.6192,-1.7715,279.8710,279.5812,278.8286,"7,477,069.5000","7,520,687.5000","7,578,839.5000","7,267,721.0000","7,218,057.0000","7,019,813.0000","9,321,300.0000","9,391,159.0000","9,445,564.0000",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,"94,701.8750","94,149.6250","92,513.4375",71.0432,69.7245,70.3611,0.3154,0.3342,0.3693,0.8628,0.4722,0.4350,-0.5147,-0.4115,-0.3975,284.1389,283.9165,283.0750,"11,285,847.0000","11,225,345.0000","11,226,525.0000","13,613,786.0000","13,532,295.0000","13,232,039.0000","13,247,678.0000","13,289,360.0000","13,230,410.0000",0.0000,0.0000,0.0000,0.0021,0.0025,0.0030,0.0001,0.0001,0.0002
75%,"95,144.1250","94,587.8438","92,930.6562",85.2612,84.0130,84.9329,0.8592,0.8581,0.8830,2.6371,2.2466,2.0376,0.8100,0.9877,1.1027,289.5986,289.2996,288.4045,"17,357,920.0000","17,292,656.0000","17,088,208.0000","19,817,088.0000","19,762,840.0000","19,424,888.0000","19,746,144.0000","19,837,912.0000","19,761,216.0000",0.0002,0.0003,0.0003,0.0289,0.0311,0.0322,0.0066,0.0070,0.0076
max,"96,317.1250","95,743.3750","94,034.6250",101.7909,101.7601,102.8384,1.0000,1.0000,1.0000,13.0567,12.1859,11.5511,8.9799,9.5909,10.7110,310.0623,309.3418,308.5383,"34,185,728.0000","34,257,408.0000","34,426,112.0000","34,175,376.0000","34,015,120.0000","33,640,528.0000","37,201,360.0000","37,450,896.0000","37,722,000.0000",0.0473,0.0503,0.0560,2.0690,1.8669,1.8503,0.6681,0.7031,0.7346


In [27]:
from IPython.core.display import display

In [30]:
df_solar_meteo[col].value_counts().sort_values(ascending=False).iloc[:3]

0.0000    6108
0.0000      53
0.0000      46
Name: (3, VAR79), dtype: int64

In [34]:
for df in [df_solar_meteo, df_solar_power]:
    for col in df:
        tmpdf = pd.DataFrame(df_solar_meteo[col].value_counts().sort_values(ascending=False).iloc[:3])
        if tmpdf.iloc[0, 0] > 100:
            display(tmpdf)

,1
,VAR164
0.0000,3094
1.0000,569
0.0017,90


,2
,VAR164
0.0000,2976
1.0000,585
0.0021,102


,3
,VAR164
0.0000,2722
1.0000,616
0.0025,95


,1
,VAR228
0.0000,10154
0.0001,22
0.0001,21


,2
,VAR228
0.0000,9989
0.0005,20
0.0002,20


...

,3
,VAR78
0.0000,4008
0.0001,107
0.0002,65


,1
,VAR79
0.0000,6531
0.0000,42
0.0000,38


,2
,VAR79
0.0000,6344
0.0000,44
0.0000,40


,3
,VAR79
0.0000,6108
0.0000,53
0.0000,46


AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [20]:
df_solar_power.value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'